In [1]:
#Run Packages
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd
from pandas import json_normalize
import sys

In [10]:
#User Input for a Stock using input() command
stock=input()

 amzn


In [11]:
#The function below takes two inputs (urls for two different API calls) and returns all the requested information about a given stock. If the url/API call malfunctions or the user inputs a stock that does not exist, the function also handles errors.
def get_api_stock_data(url, url2):
    try:
        response = requests.get(url, headers=headers, params=querystring)
        r = requests.get(url2, headers=headers, params={'modules': 'financialData'})
        response.raise_for_status()
        r.raise_for_status()
    
    except requests.exceptions.HTTPError as errh: #Handles API errors
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)
    
    try:
        symbol_json = response.json() 
        print(" Name Ticker: " + symbol_json['quoteResponse']['result'][0]['symbol'])
        companyname_json = response.json() 
        print(" Company Name: " + companyname_json['quoteResponse']['result'][0]["longName"])
        stock_json = response.json() 
        print(" Current Price: $" + str(stock_json['quoteResponse']['result'][0]["regularMarketPrice"]))
        targetmeanprice_json = r.json()
        print(" Target Mean Price: $" + str(targetmeanprice_json['quoteSummary']['result'][0]['financialData']['targetMeanPrice']['fmt']))
        cashonhand_json = r.json()
        print(" Cash On Hand: $" + str(cashonhand_json['quoteSummary']['result'][0]['financialData']['freeCashflow']['fmt']))
        profitmargins_json = r.json()
        print(" Profit Margins: " + str(profitmargins_json['quoteSummary']['result'][0]['financialData']['profitMargins']['fmt']))
        
    except IndexError: #Handles an error in which the stock does not exist (user error)
        return "Stock does not exist"

In [12]:
symbol = stock
apikey="6D1NnQnQNQ6JXgG4KFmXR5EfP3wtEqJG3xgAto1X"
url = url = "https://yfapi.net/v6/finance/quote"
url2 = "https://yfapi.net/v11/finance/quoteSummary/{}".format(symbol)
querystring = {"symbols":stock}
headers = {
    'x-api-key': apikey
    }

get_api_stock_data(url,url2)

 Name Ticker: AMZN
 Company Name: Amazon.com, Inc.
 Current Price: $93.76
 Target Mean Price: $133.64
 Cash On Hand: $7.62M
 Profit Margins: -0.53%


In [13]:
#Store Data in DataFrame Format AND Handle Errors
def get_api_response(url, url2, response_type):
    try:
        response = requests.get(url, headers=headers, params=querystring)
        r = requests.get(url2, headers=headers, params={'modules': 'financialData'})
        response.raise_for_status()
        r.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)
    
    if response_type == 'dataframe':
        #v6 information
        dictr = response.json()
        stockdf = dictr['quoteResponse']['result']
        df = pd.json_normalize(stockdf)
        smalldf = df[["symbol", "longName", "regularMarketPrice"]]
        #v11 information
        dictr2 = r.json()
        stockdf2 = dictr2['quoteSummary']['result'][0]['financialData']
        df2 = pd.json_normalize(stockdf2)
        smalldf2 = df2[['targetMeanPrice.fmt', 'freeCashflow.fmt', 'profitMargins.fmt']]
        #synthesis into one dataframe
        dftogether = pd.concat([smalldf, smalldf2], axis=1)
        dftogether['Date'] = pd.Timestamp.today().strftime('%m-%d-%Y')
        dftogether.columns = dftogether.columns.astype(str)
        dftogether = dftogether.set_index('symbol', drop=False).rename_axis(None)
        dftogether.columns = ['Symbol', 'CompanyName', 'CurrentPrice','TargetMeanPrice','CashOnHand','ProfitMargins','DateofDataPulled']
        result = dftogether
    else:
        result = "An unhandled error has occurred!"
        
    return result

In [14]:
#DataFrame of Stock Information
response_type = 'dataframe' #If this input is changed, the unhandled error will result
apikey="6D1NnQnQNQ6JXgG4KFmXR5EfP3wtEqJG3xgAto1X"
url = "https://yfapi.net/v6/finance/quote"
symbol = stock
url2 = "https://yfapi.net/v11/finance/quoteSummary/{}".format(symbol)
querystring = {"symbols":stock}
headers = {
    'x-api-key': apikey
    }

finaldf = get_api_response(url, url2, response_type)
finaldf

,Symbol,CompanyName,CurrentPrice,TargetMeanPrice,CashOnHand,ProfitMargins,DateofDataPulled
AMZN,AMZN,"Amazon.com, Inc.",93.76,133.64,7.62M,-0.53%,02-27-2023


In [15]:
#JSON Format of Information (Not Indented, JSON String)
result = finaldf.to_json(orient="records")
print(result)

[{"Symbol":"AMZN","CompanyName":"Amazon.com, Inc.","CurrentPrice":93.76,"TargetMeanPrice":"133.64","CashOnHand":"7.62M","ProfitMargins":"-0.53%","DateofDataPulled":"02-27-2023"}]


In [16]:
#JSON Format of Information (Indented)
result2 = finaldf.to_json(orient="records", indent=4)
print(result2)

[
    {
        "Symbol":"AMZN",
        "CompanyName":"Amazon.com, Inc.",
        "CurrentPrice":93.76,
        "TargetMeanPrice":"133.64",
        "CashOnHand":"7.62M",
        "ProfitMargins":"-0.53%",
        "DateofDataPulled":"02-27-2023"
    }
]


In [17]:
#JSON Format of Information (Indented with Index Value)
result2 = finaldf.to_json(orient="index", indent=4)
print(result2)

{
    "AMZN":{
        "Symbol":"AMZN",
        "CompanyName":"Amazon.com, Inc.",
        "CurrentPrice":93.76,
        "TargetMeanPrice":"133.64",
        "CashOnHand":"7.62M",
        "ProfitMargins":"-0.53%",
        "DateofDataPulled":"02-27-2023"
    }
}
